In [1]:
# Imports

import json
import geojson
import geopandas
import csv
import copy
import ast
import os
from collections import OrderedDict

from geojson import Feature, Point, FeatureCollection


In [2]:
fips_codes = {
 '01': {'abbr': 'AL', 'full_name': 'alabama'},
 '02': {'abbr': 'AK', 'full_name': 'alaska'},
 '04': {'abbr': 'AZ', 'full_name': 'arizona'},
 '05': {'abbr': 'AR', 'full_name': 'arkansas'},
 '06': {'abbr': 'CA', 'full_name': 'california'},
 '08': {'abbr': 'CO', 'full_name': 'colorado'},
 '09': {'abbr': 'CT', 'full_name': 'connecticut'},
 '10': {'abbr': 'DE', 'full_name': 'delaware'},
 '11': {'abbr': 'DC', 'full_name': 'district_of_columbia'},
 '12': {'abbr': 'FL', 'full_name': 'florida'},
 '13': {'abbr': 'GA', 'full_name': 'georgia'},
 '15': {'abbr': 'HI', 'full_name': 'hawaii'},
 '16': {'abbr': 'ID', 'full_name': 'idaho'},
 '17': {'abbr': 'IL', 'full_name': 'illinois'},
 '18': {'abbr': 'IN', 'full_name': 'indiana'},
 '19': {'abbr': 'IA', 'full_name': 'iowa'},
 '20': {'abbr': 'KS', 'full_name': 'kansas'},
 '21': {'abbr': 'KY', 'full_name': 'kentucky'},
 '22': {'abbr': 'LA', 'full_name': 'louisiana'},
 '23': {'abbr': 'ME', 'full_name': 'maine'},
 '24': {'abbr': 'MD', 'full_name': 'maryland'},
 '25': {'abbr': 'MA', 'full_name': 'massachusetts'},
 '26': {'abbr': 'MI', 'full_name': 'michigan'},
 '27': {'abbr': 'MN', 'full_name': 'minnesota'},
 '28': {'abbr': 'MS', 'full_name': 'mississippi'},
 '29': {'abbr': 'MO', 'full_name': 'missouri'},
 '30': {'abbr': 'MT', 'full_name': 'montana'},
 '31': {'abbr': 'NE', 'full_name': 'nebraska'},
 '32': {'abbr': 'NV', 'full_name': 'nevada'},
 '33': {'abbr': 'NH', 'full_name': 'new_hampshire'},
 '34': {'abbr': 'NJ', 'full_name': 'new_jersey'},
 '35': {'abbr': 'NM', 'full_name': 'new_mexico'},
 '36': {'abbr': 'NY', 'full_name': 'new_york'},
 '37': {'abbr': 'NC', 'full_name': 'north_carolina'},
 '38': {'abbr': 'ND', 'full_name': 'north_dakota'},
 '39': {'abbr': 'OH', 'full_name': 'ohio'},
 '40': {'abbr': 'OK', 'full_name': 'oklahoma'},
 '41': {'abbr': 'OR', 'full_name': 'oregon'},
 '42': {'abbr': 'PA', 'full_name': 'pennsylvania'},
 '44': {'abbr': 'RI', 'full_name': 'rhode_island'},
 '45': {'abbr': 'SC', 'full_name': 'south_carolina'},
 '46': {'abbr': 'SD', 'full_name': 'south_dakota'},
 '47': {'abbr': 'TN', 'full_name': 'tennessee'},
 '48': {'abbr': 'TX', 'full_name': 'texas'},
 '49': {'abbr': 'UT', 'full_name': 'utah'},
 '50': {'abbr': 'VT', 'full_name': 'vermont'},
 '51': {'abbr': 'VA', 'full_name': 'virginia'},
 '53': {'abbr': 'WA', 'full_name': 'washington'},
 '54': {'abbr': 'WV', 'full_name': 'west_virginia'},
 '55': {'abbr': 'WI', 'full_name': 'wisconsin'},
 '56': {'abbr': 'WY', 'full_name': 'wyoming'}}

In [3]:
# Supporting Functions
# 
def read_geo_gson_file(file_path):
    with open(file_path) as f:
        gj = geojson.load(f)
        features = gj['features']
    print(f'Loaded {len(features)} features from {file_path}')
    return  features

def read_geojson_files(file_paths, base_path = "./"):
    features = []
    for file_path in file_paths:
        features += read_geo_gson_file(os.path.join(base_path, file_path))
    return features
    

def read_market_zip_code_usage(market_file_prefix, base_path="./source_data" ):
    data_type = "zip"
    csv_file_path = mapping_data_file_path(market_file_prefix, base_path=base_path, data_type=data_type)
    data_rows = []
    with open(csv_file_path, encoding='utf-8-sig') as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            data_rows.append(row)
    zip_codes = []
    for data_row in data_rows:
        zip_codes.append(data_row['Zip'].zfill(5))
    print(f'Loaded {len(zip_codes)} zip_codes from {csv_file_path}')
    return zip_codes
        
def read_market_block_group_usage(market_file_prefix, base_path="./source_data" ):
    data_type = "bg"
    csv_file_path = mapping_data_file_path(market_file_prefix, base_path=base_path, data_type=data_type)
    data_rows = []
    with open(csv_file_path, encoding='utf-8-sig') as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            data_rows.append(row)
    block_groups = []
    for data_row in data_rows:
        block_groups.append(data_row['BG'].zfill(12))
    print(f'Loaded {len(block_groups)} block groups from {csv_file_path}')
    return zip_codes
        
# market_file_prefix = 'DallasFortWorth'
# market_prefix= 'dfw'
# DallasFortWorth__Mapping_BG_Data.csv
# ./source_data/{market_file_prefix}__Mapping_BG_Data.csv - the actual zip code data
# 
# DallasFortWorth__Mapping_Zip_Data.csv
# ./source_data/{market_file_prefix}__Mapping_Zip_Data.csv - the actual block group data
# 
# DallasFortWorth__Mapping_Settings_Transposed File_BG.csv
# ./source_data/{market_file_prefix}__Mapping_Settings_Transposed_File_BG.csv - segment definitions for Block Groups
# DallasFortWorth__Mapping_Settings_Transposed File_Zip.csv
# ./source_data/{market_file_prefix}__Mapping_Settings_Transposed_File_Zip.csv - segment definitions for Zip Codes

def file_type_for_data_type(data_type = "zip"):
    if data_type == "zip": 
        return "Zip"
    if data_type == "bg":
        return "BG"

# DallasFortWorth__Mapping_BG_Data.csv
# ./source_data/{market_file_prefix}__Mapping_BG_Data.csv - the actual zip code data
# 
# DallasFortWorth__Mapping_Zip_Data.csv
# ./source_data/{market_file_prefix}__Mapping_Zip_Data.csv - the actual block group data
# 
def mapping_data_file_path(market_file_prefix, base_path="./source_data", data_type = "zip"):
    file_type = file_type_for_data_type(data_type = data_type)
    return f'{base_path}/{market_file_prefix}__Mapping_{file_type}_Data.csv'

# DallasFortWorth__Mapping_Settings_Transposed File_BG.csv
# ./source_data/{market_file_prefix}__Mapping_Settings_Transposed_File_BG.csv - segment definitions for Block Groups
# DallasFortWorth__Mapping_Settings_Transposed File_Zip.csv
# ./source_data/{market_file_prefix}__Mapping_Settings_Transposed_File_Zip.csv - segment definitions for Zip Codes
def map_settings_data_file_path(market_file_prefix, base_path="./source_data", data_type = "zip"):
    file_type = file_type_for_data_type(data_type = data_type)
    return f'{base_path}/{market_file_prefix}__Mapping_Settings_Transposed File_{file_type}.csv'

def published_map_data_path(market_prefix,  data_type = "zip", published_data_base_path="./public"):
    return f'{published_data_base_path}/{market_prefix}_{data_type}_data.json'

def published_map_settings_path(market_prefix,  data_type = "zip", published_data_base_path="./public"):
    return f'{published_data_base_path}/{market_prefix}_{data_type}_settings_data.json'

def published_map_labels_path(market_prefix,  data_type = "zip", published_data_base_path="./public"):
    return f'{published_data_base_path}/{market_prefix}_{data_type}_labels.json'

def read_mapping_data(market_file_prefix, base_path='./source_data', data_type='zip'):
    data_rows = []
    csv_file_path = mapping_data_file_path(market_file_prefix, base_path=base_path, data_type = data_type)
    with open(csv_file_path) as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            data_rows.append(row)
    print(f'Loaded {len(data_rows)} data rows from {csv_file_path}')
    return data_rows

def create_zip_features(features, zip_data_rows):
    new_features_to_keep = []
    for data_row in zip_data_rows:
        zip_code = data_row['Zip'].zfill(5)
        # find the associated feature
        for feature in features:
            if feature['properties']["ZCTA5CE10"] == zip_code:
                new_feature = copy.deepcopy(feature)
                for key in data_row.keys():
                    if key != 'Zip':
                        if data_row[key]:
                            if data_row[key] != 'inf':
                                new_feature['properties'][key] =  ast.literal_eval(data_row[key].replace(',',''))
                new_features_to_keep.append(new_feature)
                break
    print(f'Processed {len(new_features_to_keep)} features to use')
    return new_features_to_keep

def create_bg_features(features, bg_data_rows):
    new_features_to_keep = []
    for data_row in bg_data_rows:
        block_group = str(data_row['BG']).zfill(12)
        # find the associated feature
        found = False
        for feature in features:
            if feature['properties']["GEOID10"] == block_group:
                found = True
                new_feature = copy.deepcopy(feature)
                for key in data_row.keys():
                    if key != 'BG':
                        if data_row[key] and data_row[key] != 'inf':
                            new_feature['properties'][key] =  ast.literal_eval(data_row[key].replace(',',''))
                new_features_to_keep.append(new_feature)
                break
    #         if found == False:
    #             print(f'failed: {block_group}')
    print(f'Processed {len(new_features_to_keep)} features to use')
    return new_features_to_keep

def create_geojson(features):
    print(f'Created GeoJson with  {len(features)} features')
    return FeatureCollection(features)

def write_geojson(geojson_data, market_prefix, data_type='zip',published_data_base_path = "./public"):
    geojson_string = geojson.dumps(geojson_data, sort_keys=True)
    f = open(f'{published_data_base_path}/{market_prefix}_{data_type}_data.json', "w")
    f.write(geojson_string)
    f.close()
    print(f'Wrote geojson data to  {published_data_base_path}/{market_prefix}_{data_type}_data.json')
    
def read_map_settings(market_file_prefix, base_path="./source_data", data_type = "zip"):
    data_rows = []
    csv_file_path = map_settings_data_file_path(market_file_prefix, base_path=base_path, data_type = data_type)
    with open(csv_file_path, encoding='utf-8-sig') as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            data_rows.append(row)
    print(f'Loaded {len(data_rows)} data rows from {csv_file_path}')   
    return data_rows

def write_settings_data(settings_data, market_file_prefix, base_path="./public", data_type = "zip"):
    processed_settings_data = OrderedDict()
    for row in settings_data:
        processed_settings_data[row['Bain_Short_Name']] = row
    json_string = json.dumps(processed_settings_data, indent = 2, sort_keys=False)
    settings_data_file_path = published_map_settings_path(market_file_prefix,  data_type = data_type, published_data_base_path=base_path)
    f = open(settings_data_file_path, "w")
    f.write(json_string)
    f.close()
    print(f'Wrote settings data rows to {settings_data_file_path}') 
    
def create_zip_labels(market_file_prefix, base_path="./source_data" , data_type = "zip" , published_data_base_path="./public"):
    data_rows = []
    #     csv_file_path = './DFW_Mapping_data.csv'
    csv_file_path = mapping_data_file_path(market_file_prefix, base_path=base_path, data_type = "zip")
    with open(csv_file_path, encoding='utf-8-sig') as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            data_rows.append(row)

    features = []
    for data_row in data_rows:
        feature =  { 
            "type": "Feature", 
            "properties": {
                "id": data_row['Zip']
            },
            "geometry": { 
                "type": "Point", 
                "coordinates": [ float(data_row['intptlong']), float(data_row['intptlat']) ]
            }
        }

        features.append(feature)
    geo_json = {
        "type": "FeatureCollection",
        "features": features
    }
    json_string = json.dumps(geo_json, indent = 2, sort_keys=True)
    output_file_path = published_map_labels_path(market_prefix,  data_type , published_data_base_path="./public")
    f = open(output_file_path, "w")
    f.write(json_string)
    f.close()
    print(f'Wrote geojson labels data to  {output_file_path}')

def create_bg_labels(market_file_prefix, base_path="./source_data" , data_type = "bg" , published_data_base_path="./public"):
    data_rows = []
    #     csv_file_path = './DFW_Mapping_data.csv'
    csv_file_path = mapping_data_file_path(market_file_prefix, base_path=base_path, data_type = "bg")
    with open(csv_file_path, encoding='utf-8-sig') as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            data_rows.append(row)

    features = []
    for data_row in data_rows:
        feature =  { 
            "type": "Feature", 
            "properties": {
                "id": data_row['BG']
            },
            "geometry": { 
                "type": "Point", 
                "coordinates": [ float(data_row['LONG']), float(data_row['LAT']) ]
            }
        }

        features.append(feature)
    geo_json = {
        "type": "FeatureCollection",
        "features": features
    }
    json_string = json.dumps(geo_json, indent = 2, sort_keys=True)
    output_file_path = published_map_labels_path(market_prefix,  data_type , published_data_base_path="./public")
    f = open(output_file_path, "w")
    f.write(json_string)
    f.close()
    print(f'Wrote geojson labels data to  {output_file_path}')
    

In [4]:
# Create all of the data
source_data_base_path = "./source_data"
source_data_base_path = "/Users/alanmccann/Dropbox/bain/23rd_run"
published_data_base_path = "./public"
geojson_files_base_path = '/Users/alanmccann/Dropbox/bain/map_source_data'
census_year = 2010
csv_file_path = f'{source_data_base_path}/Market_Mapping_Settings.csv'
market_rows = []
with open(csv_file_path, encoding='utf-8-sig') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        market_rows.append(row)
market_count = 0
completed_markets = []
for market_data in market_rows[85:90]:
    print(market_data)
    market_file_prefix = market_data['MarketName']
    market_prefix = market_file_prefix
    # Zip Processing     
    data_type = 'zip'
    print(f'market: {market_prefix}')
    geojson_files = zip_geojson_files_for_market(market_data)
    zip_codes = read_market_zip_code_usage(market_file_prefix, base_path=source_data_base_path )
    zip_data_rows = read_mapping_data(market_file_prefix, source_data_base_path, data_type)
    features = read_geojson_files(geojson_files, base_path=geojson_files_base_path)
    features_to_use = create_zip_features(features, zip_data_rows)
    new_geojson = create_geojson(features_to_use)
    write_geojson(new_geojson, market_prefix, data_type, published_data_base_path=published_data_base_path)
    map_settings_data = read_map_settings(market_file_prefix, base_path=source_data_base_path, data_type = "zip")
    write_settings_data(map_settings_data, market_prefix, base_path=published_data_base_path, data_type = "zip")
    create_zip_labels(market_file_prefix, base_path=source_data_base_path , data_type = "zip" , published_data_base_path=published_data_base_path)
    # Block Group Processing
    data_type = 'bg'
    geojson_files = bg_geojson_files_for_market(market_data,census_year )
    block_groups = read_market_block_group_usage(market_file_prefix, base_path=source_data_base_path )
    block_group_rows = read_mapping_data(market_file_prefix, source_data_base_path, data_type)
    features = read_geojson_files(geojson_files, base_path=geojson_files_base_path)
    features_to_use = create_bg_features(features, block_group_rows)
    new_geojson = create_geojson(features_to_use)
    write_geojson(new_geojson, market_prefix, data_type, published_data_base_path=published_data_base_path)
    map_settings_data = read_map_settings(market_file_prefix, base_path=source_data_base_path, data_type = "bg")
    write_settings_data(map_settings_data, market_prefix, base_path=published_data_base_path, data_type = "bg")
    create_bg_labels(market_file_prefix, base_path=source_data_base_path , data_type = "bg" , published_data_base_path=published_data_base_path)
    market_count += 1
# Finished states - create overall settings file    
create_geojson_settings(csv_file_path, published_data_base_path = "./public")
print(f'*** COMPLETED {} "


market: charleston
['sc_south_carolina_zip_codes_geo.min.json']
Loaded 44 zip_codes from /Users/alanmccann/Dropbox/bain/22nd_run/CharlestonSC__Mapping_Zip_Data.csv
Loaded 44 data rows from /Users/alanmccann/Dropbox/bain/22nd_run/CharlestonSC__Mapping_Zip_Data.csv
Loaded 424 features from /Users/alanmccann/Dropbox/bain/map_source_data/sc_south_carolina_zip_codes_geo.min.json


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



Loaded 3059 features from /Users/alanmccann/Dropbox/bain/map_source_data/tl_2010_45_bg10.json
Processed 397 features to use
Created GeoJson with  397 features
Wrote geojson data to  ./public/charleston_bg_data.json
Loaded 38 data rows from /Users/alanmccann/Dropbox/bain/22nd_run/CharlestonSC__Mapping_Settings_Transposed File_BG.csv
Wrote settings data rows to ./public/charleston_bg_settings_data.json
Wrote geojson labels data to  ./public/charleston_bg_labels.json
market: dfw
['tx_texas_zip_codes_geo.min.json']
Loaded 255 zip_codes from /Users/alanmccann/Dropbox/bain/22nd_run/DallasFortWorth__Mapping_Zip_Data.csv
Loaded 255 data rows from /Users/alanmccann/Dropbox/bain/22nd_run/DallasFortWorth__Mapping_Zip_Data.csv
Loaded 1939 features from /Users/alanmccann/Dropbox/bain/map_source_data/tx_texas_zip_codes_geo.min.json


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



Processed 255 features to use
Created GeoJson with  255 features
Wrote geojson data to  ./public/dfw_zip_data.json
Loaded 38 data rows from /Users/alanmccann/Dropbox/bain/22nd_run/DallasFortWorth__Mapping_Settings_Transposed File_Zip.csv
Wrote settings data rows to ./public/dfw_zip_settings_data.json
Wrote geojson labels data to  ./public/dfw_zip_labels.json
Loaded 4063 block groups from /Users/alanmccann/Dropbox/bain/22nd_run/DallasFortWorth__Mapping_BG_Data.csv
Loaded 4063 data rows from /Users/alanmccann/Dropbox/bain/22nd_run/DallasFortWorth__Mapping_BG_Data.csv
Loaded 15811 features from /Users/alanmccann/Dropbox/bain/map_source_data/tl_2010_48_bg10.json
Processed 4063 features to use
Created GeoJson with  4063 features
Wrote geojson data to  ./public/dfw_bg_data.json
Loaded 38 data rows from /Users/alanmccann/Dropbox/bain/22nd_run/DallasFortWorth__Mapping_Settings_Transposed File_BG.csv
Wrote settings data rows to ./public/dfw_bg_settings_data.json
Wrote geojson labels data to  ./

...

Processed 189 features to use
Created GeoJson with  189 features
Wrote geojson data to  ./public/tampa_zip_data.json
Loaded 38 data rows from /Users/alanmccann/Dropbox/bain/22nd_run/Tampa__Mapping_Settings_Transposed File_Zip.csv
Wrote settings data rows to ./public/tampa_zip_settings_data.json
Wrote geojson labels data to  ./public/tampa_zip_labels.json
Loaded 2653 block groups from /Users/alanmccann/Dropbox/bain/22nd_run/Tampa__Mapping_BG_Data.csv
Loaded 2653 data rows from /Users/alanmccann/Dropbox/bain/22nd_run/Tampa__Mapping_BG_Data.csv
Loaded 11442 features from /Users/alanmccann/Dropbox/bain/map_source_data/tl_2010_12_bg10.json
Processed 2653 features to use
Created GeoJson with  2653 features
Wrote geojson data to  ./public/tampa_bg_data.json
Loaded 38 data rows from /Users/alanmccann/Dropbox/bain/22nd_run/Tampa__Mapping_Settings_Transposed File_BG.csv
Wrote settings data rows to ./public/tampa_bg_settings_data.json
Wrote geojson labels data to  ./public/tampa_bg_labels.json
ma

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



Processed 1879 features to use
Created GeoJson with  1879 features
Wrote geojson data to  ./public/texas_zip_data.json
Loaded 38 data rows from /Users/alanmccann/Dropbox/bain/22nd_run/Texas__Mapping_Settings_Transposed File_Zip.csv
Wrote settings data rows to ./public/texas_zip_settings_data.json
Wrote geojson labels data to  ./public/texas_zip_labels.json
Loaded 15811 block groups from /Users/alanmccann/Dropbox/bain/22nd_run/Texas__Mapping_BG_Data.csv
Loaded 15811 data rows from /Users/alanmccann/Dropbox/bain/22nd_run/Texas__Mapping_BG_Data.csv
Loaded 15811 features from /Users/alanmccann/Dropbox/bain/map_source_data/tl_2010_48_bg10.json
Processed 15811 features to use
Created GeoJson with  15811 features
Wrote geojson data to  ./public/texas_bg_data.json
Loaded 38 data rows from /Users/alanmccann/Dropbox/bain/22nd_run/Texas__Mapping_Settings_Transposed File_BG.csv
Wrote settings data rows to ./public/texas_bg_settings_data.json
Wrote geojson labels data to  ./public/texas_bg_labels.j

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



Wrote geojson data to  ./public/waco_zip_data.json
Loaded 38 data rows from /Users/alanmccann/Dropbox/bain/22nd_run/Waco__Mapping_Settings_Transposed File_Zip.csv
Wrote settings data rows to ./public/waco_zip_settings_data.json
Wrote geojson labels data to  ./public/waco_zip_labels.json
Loaded 193 block groups from /Users/alanmccann/Dropbox/bain/22nd_run/Waco__Mapping_BG_Data.csv
Loaded 193 data rows from /Users/alanmccann/Dropbox/bain/22nd_run/Waco__Mapping_BG_Data.csv
Loaded 15811 features from /Users/alanmccann/Dropbox/bain/map_source_data/tl_2010_48_bg10.json
Processed 193 features to use
Created GeoJson with  193 features
Wrote geojson data to  ./public/waco_bg_data.json
Loaded 38 data rows from /Users/alanmccann/Dropbox/bain/22nd_run/Waco__Mapping_Settings_Transposed File_BG.csv
Wrote settings data rows to ./public/waco_bg_settings_data.json
Wrote geojson labels data to  ./public/waco_bg_labels.json
DONE
